In [ ]:
# Second Experiment for DrunkerNot

import pathlib
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow import keras
import tensorflow as tf
import PIL
import os
import numpy as np
import keras_tuner as kt
import matplotlib.pyplot as plt
PROJECT_ROOT_DIR = "."
data_dir = os.path.join(PROJECT_ROOT_DIR, "TrainPhotos")
data_dir2 = os.path.join(PROJECT_ROOT_DIR, "ValPhotos")

data_dir = pathlib.Path(data_dir)
data_dir2 = pathlib.Path(data_dir2)
print(data_dir)

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

# Set Batch size to be all images as we don't care about splitting into batches right now
batch_size = image_count
img_height = 364
img_width = 364


train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir2,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

# Get x/y train and x/y test
for image_batch, labels_batch in train_ds:
    x_train = image_batch.numpy()
    y_train = labels_batch.numpy()
    break

for image_batch, labels_batch in val_ds:
    x_test = image_batch.numpy()
    y_test = labels_batch.numpy()
    break

# Divide by 255.0 to reduce the color
x_train, x_test = x_train / 255.0, x_test / 255.0
print(x_train.shape)
print(x_test.shape)

# plt.figure(figsize=(10, 10))
# for i in range(25):
#     plt.subplot(5, 5, i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(x_train[i])
#     plt.xlabel(class_names[y_train[i]])
# plt.show()

# Model Builder for the tuner


def model_builder(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu',
              input_shape=(364, 364, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
                  metrics=['accuracy'])
    return model


# Tuner to find the best hyperparameters
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory=data_dir)


stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train, y_train, epochs=50,
             validation_split=0.2, callbacks=[stop_early])

# Get Best hyperparameters from tuner
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

# Plot graph of best hyperparamters and the accuracy of each epoch
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label='val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')
# plt.show()

# Evaluate model on unseen test data
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

# Print the accuracy on the unseen test data
print("Accuracy on Test Data: ", test_acc)